In [ ]:
#r "nuget: ModelContextProtocol, 0.4.0-preview.3"

#r "nuget: Microsoft.SemanticKernel, 1.67.1"
#r "nuget: Microsoft.SemanticKernel.Agents.Core, 1.67.1"

Installed Packages Microsoft.SemanticKernel, 1.67.1 Microsoft.SemanticKernel.Agents.Core, 1.67.1 ModelContextProtocol, 0.4.0-preview.3

In [3]:


#!import config/Settings.cs

using Microsoft.SemanticKernel;
using Kernel = Microsoft.SemanticKernel.Kernel;

Kernel CreateKernel()
{
    var builder = Kernel.CreateBuilder();

    var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId, embeddingEndpoint, embeddingApiKey) = Settings.LoadFromFile();
    
    builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
    var kernel = builder.Build();

    return kernel;
}

var kernel = CreateKernel();

# MCP Client

In [4]:
using ModelContextProtocol.Client;

In [5]:
// https://github.com/modelcontextprotocol/servers-archived/tree/main/src/github

var mcpClient = await McpClient.CreateAsync(
    new StdioClientTransport(new()
        {
            Name = "MCPServer",
            Command = "npx",
            Arguments = ["-y", "@modelcontextprotocol/server-github"],

            // EnvironmentVariables = new Dictionary<string, string>
            // {
            //     ["GITHUB_PERSONAL_ACCESS_TOKEN":] = "XXXXXXXX"             
            // }
        }));

// Retrieve the list of tools available on the MCP server
var tools = await mcpClient.ListToolsAsync().ConfigureAwait(false);

In [6]:
#pragma warning disable SKEXP0001

kernel.Plugins.AddFromFunctions("github", tools.Select(aiFunction => aiFunction.AsKernelFunction()));

In [7]:
using Microsoft.SemanticKernel.Agents;

var githubAgent = new ChatCompletionAgent
{
    Name = "GithubAgent",
    Instructions = 
        """
        You are an agent that helps questions about github
        """,
    Description = "An github assistent",
    Kernel = kernel,


    // Instruct SK to allow the function calling
    Arguments = new KernelArguments(new PromptExecutionSettings{ FunctionChoiceBehavior = FunctionChoiceBehavior.Auto(),})
};

In [8]:
var response = githubAgent.InvokeAsync("list the latest commits of the semantic-kernel repo?");

await foreach (var item in response)
{
    Console.WriteLine(item.Message);
}

Here are the latest commits in the Microsoft Semantic Kernel repository:

1. **Commit: [43da976](https://github.com/microsoft/semantic-kernel/commit/43da976c9b42548088e51e326b7df90bdc57a056)**
   - **Author:** [Mark Wallace](https://github.com/markwallace-microsoft)
   - **Message:** *Bump react and react-dom versions to be 19.2.1.*
   - **Date:** Dec 4, 2025

2. **Commit: [4ac3767](https://github.com/microsoft/semantic-kernel/commit/4ac376749dcc98ac8f9b0081e574b0d34684a35f)**
   - **Author:** [Shay Rojansky](https://github.com/roji)
   - **Message:** *Fix C# 14 Contains translation for MEVD providers.*
   - **Date:** Dec 4, 2025

3. **Commit: [0195778](https://github.com/microsoft/semantic-kernel/commit/0195778be25892cba507fe593d5592b71c2adf68)**
   - **Author:** [Ben Thomas](https://github.com/alliscode)
   - **Message:** *Updating SECURITY.ms to latest version.*
   - **Date:** Dec 3, 2025

4. **Commit: [9ee675a](https://github.com/microsoft/semantic-kernel/commit/9ee675aa66e899ca936

MCP Remote server

In [9]:
//Trick to resolve the issue with DLL loading
{
    var t=typeof(System.Net.ServerSentEvents.SseFormatter);
    Console.WriteLine(t.Assembly.FullName);

    AppDomain.CurrentDomain.AssemblyResolve += (sender, args) =>
    {
        
        //Console.WriteLine($"AssemblyResolve {args.Name}");

        // If the missing assembly matches something you're expecting
        if (args.Name.StartsWith("System.Net.ServerSentEvents"))
        {

            return t.Assembly;
        }

        return null; // Let the runtime continue trying otherwise
    };
}


System.Net.ServerSentEvents, Version=10.0.0.0, Culture=neutral, PublicKeyToken=cc7b13ffcd2ddd51


In [10]:
var mcpClient = await McpClient.CreateAsync(
    new HttpClientTransport(
        new HttpClientTransportOptions()
        { 
            Endpoint = new Uri("https://learn.microsoft.com/api/mcp") 
        }));

// Retrieve the list of tools available on the MCP server
var tools = await mcpClient.ListToolsAsync().ConfigureAwait(false);

In [11]:
#pragma warning disable SKEXP0001

kernel.Plugins.AddFromFunctions("MSDocs", tools.Select(aiFunction => aiFunction.AsKernelFunction()));

In [12]:
using Microsoft.SemanticKernel.Agents;

var microsoftDevAgent = new ChatCompletionAgent
{
    Name = "MSDevAgent",
    Instructions = 
        """
        You are a software developer that profession in Microsoft technologies
        """,
    Description = "A Software Dev",
    Kernel = kernel,


    // Instruct SK to allow the function calling
    Arguments = new KernelArguments(new PromptExecutionSettings{ FunctionChoiceBehavior = FunctionChoiceBehavior.Auto(),})
};

var response = microsoftDevAgent.InvokeAsync("which nuget do you need to add to create chat completion agent with semantic kernel?");

await foreach (var item in response)
{
    Console.WriteLine(item.Message);
}

To create a `ChatCompletionAgent` with the Semantic Kernel in .NET, you need to reference the following NuGet packages:

1. **[Microsoft.SemanticKernel](https://www.nuget.org/packages/Microsoft.SemanticKernel)**: Contains the core Semantic Kernel libraries for getting started with the Agent Framework.
2. **[Microsoft.SemanticKernel.Agents.Core](https://www.nuget.org/packages/Microsoft.SemanticKernel.Agents.Core)**: Includes the `ChatCompletionAgent`.
3. **Any relevant AI service-specific package**, such as:
   - **[Microsoft.SemanticKernel.Agents.OpenAI](https://www.nuget.org/packages/Microsoft.SemanticKernel.Agents.OpenAI)**: Provides tools to interact with OpenAI Assistant API.

These packages allow you to integrate the `ChatCompletionAgent` with the services of your choice, like Azure OpenAI or OpenAI.

You may start installing the packages using the .NET CLI:
```powershell
dotnet add package Microsoft.SemanticKernel --prerelease
dotnet add package Microsoft.SemanticKernel.Agents.Co

# Adding your dotnet tool as MCP Server

MCP Server using stdio: [..\src\SKCodeAssistent\FirstMCPServer](..\src\SKCodeAssistent\FirstMCPServer\Program.cs)

In [13]:
Console.WriteLine(Environment.CurrentDirectory);

c:\Users\tamirdresher\source\repos\creating-ai-agents-with-csharp\notebooks


In [14]:
#pragma warning disable SKEXP0001

var kernel = CreateKernel();

var mcpClient = await McpClient.CreateAsync(
    new StdioClientTransport(new()
        {
            Name = "MyFirstLocalMCP",
            Command = "dotnet",
            Arguments = ["run", "--project", "..\\src\\SKCodeAssistent\\FirstMCPServer\\FirstMCPServer.csproj"],
        }));

// Retrieve the list of tools available on the MCP server
var tools = await mcpClient.ListToolsAsync().ConfigureAwait(false);

kernel.Plugins.AddFromFunctions("myMcp", tools.Select(aiFunction => aiFunction.AsKernelFunction()));


PromptExecutionSettings settings = new() { FunctionChoiceBehavior = FunctionChoiceBehavior.Auto() };
var response = await kernel.InvokePromptAsync("echo my name: Tamir Dresher", new KernelArguments(settings));

Console.WriteLine(response);

Here are the echoed results for your name:

- Normal echo: **Hello from C#: Tamir Dresher**
- Reverse echo: **rehserD rimaT**


# Extra: MCP Server

First, install the base templates

```shell
dotnet new install Microsoft.Extensions.AI.Templates
```

then, create you MCP Server project

```shell
dotnet new mcpserver -n SampleMcpServer
cd SampleMcpServer
dotnet build
```

This will create a starting project which uses the `ModelContextProtocol` C# SDK

## Add you tools by decorating your class methods with the `[McpServerTool]` attribute

In [15]:
using ModelContextProtocol.Server;
using System.ComponentModel;

[McpServerTool]
[Description("Describes random weather in the provided city.")]
public string GetCityWeather(
    [Description("Name of the city to return weather for")] string city)
{
    var weather = "balmy,rainy,stormy";    

    var weatherChoices = weather.Split(",");
    var selectedWeatherIndex =  Random.Shared.Next(0, weatherChoices.Length);

    return $"The weather in {city} is {weatherChoices[selectedWeatherIndex]}.";
}

## Publishing to nuget
follow the steps here:
https://devblogs.microsoft.com/dotnet/mcp-server-dotnet-nuget-quickstart/#📋-configuring-for-nuget-publication

Nuget is able to filter for packages with MCP Servers
![image.png](./img/nuget-mcp-search.png)

## Consuming a nuget based MCP Servers

From net10, the SDK supports the `dnx` command (like npx for dotnet)

Simply put this in your MCP Client config

```json
{
  "inputs": [],
  "servers": {
    "mcpserver.everything.stdio": {
      "type": "stdio",
      "command": "dnx",
      "args": ["YOUR_NUGET_PACKAGE@0.4.0", "--yes"],
      "env": {}
    }
  }
}
```

![image.png](./img/nuget-mcp-use-instructions.png)